# Лекция №2

## На прошлой лекции

1. [О чем курс?](../lecture_1/lecture_1.ipynb#what)
2. [Для кого?](../lecture_1/lecture_1.ipynb#who)
3. [Литература + ПО?](../lecture_1/lecture_1.ipynb#lib_soft)
4. [Ansys vs OpenFOAM. Кто твой БРО?](../lecture_1/lecture_1.ipynb#avso)
5. [Что такое OpenFOAM?](../lecture_1/lecture_1.ipynb#OpenFOAM_w)
6. [Hello world!](../lecture_1/lecture_1.ipynb#hello)
7. [Doxygen](../lecture_1/lecture_1.ipynb#Doxygen)
8. [Контейнеры?](../lecture_1/lecture_1.ipynb#Containers)
9. [Примеры простых программ](../lecture_1/lecture_1.ipynb#tests)

## План

0. [Работа с сервером](#server)
1. [Работа на кластере. Планировщик задач Slurm](#slurm)
2. [Интерполяция и argList](#Interpolation)
3. [Примитивы OpenFOAM](#src/OpenFOAM)
4. [Класс Time](#Time)
5. [Формат сетки OpenFOAM](#mesh)
6. [Заключение](#finish)

In [ ]:
# материалы можно смотреть в jupyter, VS code или ... ipynb
# ! команда - bash команды 
# все примеры можно запускать в терминале отдельно от лекций
!ls

## Работа с сервером <a name="server"></a>

[Введение в Линукс](https://stepik.org/course/73/promo) - видео лекций по работе с linux системой для тех, у кого проблемы с работой в терминале

Всем на почту пришло сообщение:

| логин сервера       | пароль           | хост  | логин кластера МФТИ      | пароль           | хост кластера МФТИ |
| ------------- |:-------------:| :-------------:| :-------------:| :-------------:| :-------------:| 
|user21	|******** |	10.55.229.140|	kurashov.vv09|	*********	|calc.cod.phystech.edu|

Сервер 
* 2 ядра, мало памяти, только для просмотра материалов!
* только из локальной сети МФТИ, нет подключения к интернету
* установлен OpenFOAM
* лекции 2022 в папке /home/lectures (git clone /home/lectures)
* Python+jupyter расширение для VS code в папке /home/lectures/vs_extensions

Кластер МФТИ
* много ядер, много памяти
* ЗАПУСКАТЬ РАСЧЕТЫ ТОЛЬКО ЧЕРЕЗ SLURM!!!

### SSH key

* [Генерация ssh ключа](https://selectel.ru/blog/tutorials/how-to-generate-ssh/) - нужно для входа без пароля
* ssh-copy-id логин@хост (пример: ssh-copy-id user1@10.55.229.140 или ssh-copy-id kurashov.vv09@calc.cod.phystech.edu), дальше ввести один раз пароль

### VS code

* установить расширение Remote-ssh
* добавить серверы
* установить на локальном сервере расширения (... + /home/lectures/vs_extensions)
* создать папку 2023/Фамилия
* сделать git clone лекций из репозитория /home/lectures

## Работа на кластере. Планировщик задач Slurm <a name="slurm"></a>

[Документация slurm](https://slurm.schedmd.com/documentation.html)

[Slurm tutorial](https://slurm.schedmd.com/tutorials.html)

[Шпаргалка slurm](../books/summary.pdf)

<img src="img/cluster.png" width="600">


# УПРАВЛЯЮЩИЙ УЗЕЛ ОДИН ДЛЯ ВСЕХ! НУЖЕН ТОЛЬКО ДЛЯ УПРАВЛЕНИЯ!

# НИКОГДА НЕ ЗАПУСКАТЬ БОЛЬШИЕ ЗАДАЧИ НА УПРАВЛЯЮЩЕМ УЗЛЕ!

### Основные команды:

* sacct is used to report job or job step accounting information about active or completed jobs.

* salloc is used to allocate resources for a job in real time. Typically this is used to allocate resources and spawn a shell. The shell is then used to execute srun commands to launch parallel tasks.

* sattach is used to attach standard input, output, and error plus signal capabilities to a currently running job or job step. One can attach to and detach from jobs multiple times.

* sbatch is used to submit a job script for later execution. The script will typically contain one or more srun commands to launch parallel tasks.

* sbcast is used to transfer a file from local disk to local disk on the nodes allocated to a job. This can be used to effectively use diskless compute nodes or provide improved performance relative to a shared file system.

* scancel is used to cancel a pending or running job or job step. It can also be used to send an arbitrary signal to all processes associated with a running job or job step.

* scontrol is the administrative tool used to view and/or modify Slurm state. Note that many scontrol commands can only be executed as user root.

* sinfo reports the state of partitions and nodes managed by Slurm. It has a wide variety of filtering, sorting, and formatting options.

* sprio is used to display a detailed view of the components affecting a job's priority.

* squeue reports the state of jobs or job steps. It has a wide variety of filtering, sorting, and formatting options. By default, it reports the running jobs in priority order and then the pending jobs in priority order.

* srun is used to submit a job for execution or initiate job steps in real time. srun has a wide variety of options to specify resource requirements, including: minimum and maximum node count, processor count, specific nodes to use or not use, and specific node characteristics (so much memory, disk space, certain required features, etc.). A job can contain multiple job steps executing sequentially or in parallel on independent or shared resources within the job's node allocation.

* sshare displays detailed information about fairshare usage on the cluster. Note that this is only viable when using the priority/multifactor plugin.

* sstat is used to get information about the resources utilized by a running job or job step.

* strigger is used to set, get or view event triggers. Event triggers include things such as nodes going down or jobs approaching their time limit.

* sview is a graphical user interface to get and update state information for jobs, partitions, and nodes managed by Slurm.

### Примеры:
для удобства удаленный запуск через ssh:
прописать в ~/.bashrc функцию

```bash
phystech()
{
    # ssh USER@HOST command
    ssh kurashov.vv01@calc.cod.phystech.edu $1
}
```

phystech ls - запускает команду ls на HOST под USER

In [ ]:
# управляющий узел
phystech = "ssh kurashov.vv01@calc.cod.phystech.edu "

In [ ]:
# ресурсы управляющего узла
!$phystech "lscpu"

In [ ]:
# ресурсы одного из расчетных узлов
# srun - запуск текущей или начальной комманды на вычислительном узле
# для параллельного расчета используем sbatch
# у каждой задачи есть id

!$phystech "srun lscpu"

In [ ]:
# sinfo - текущее состояние ресурсов
# idle - доступно
# alloc - занято полностью

!$phystech "sinfo"

In [ ]:
# squeue - текущее состояние очереди задач

!$phystech "squeue"

In [ ]:
# sacct - журнал выполнения задач

!$phystech "sacct"

In [ ]:
#sbatch - добавление задачи в очередь (задача - bash скрипт с опциями)

!$phystech "cat example_run.sh"

пример задачи из текстового файла example_run.sh
```bash

#!/bin/bash
#SBATCH -n 16
#SBATCH --partition=RT
#SBATCH --job-name=sleep
#SBATCH --comment="run sleep example for education"

sleep 60

# OpenFOAM run
# decomposePar
# mpirun -np 16 foam_solver -parallel

```

#SBATCH - опции запуска

sleep 60 - запуск команды sleep с опцией 60


In [ ]:
from time import sleep
# проверяем доступные ресурсы
!$phystech "sinfo"

# добавляем задачу в очередь
!$phystech "sbatch example_run.sh"

# еще раз смотрим на ресурсы. Убеждаемся, что нам дали узел (16 ядер, пол узла) для расчета
!$phystech "sinfo"

# Проверяем текущий статус задачи в очереди задач. Убеждаемся, что наша задача запущена
!$phystech "squeue"

sleep(60) # ждем минуту

# Проверяем текущий статус задачи в очереди задач. Очередь должна быть пуста, задача исполнена
!$phystech "squeue"

# Смотрит отметку о выполнении задачи в журнале задач
!$phystech "sacct"

In [ ]:
# лог работы записывается в файл slurm-JOB_ID.out
!$phystech ls

In [ ]:
# scancel - отменяет задачу по id

## Интерполяция и аргументы командной строки<a name="Interpolation"></a>

In [ ]:
%cd Interpolation

In [ ]:
!ls

In [ ]:
# отдельная папка для запуска
!ls test

In [ ]:
#!cat Interpolation.C

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | Copyright (C) 2011-2016 OpenFOAM Foundation
     \\/     M anipulation  |
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

Application
    Interpolation


Description
   Interpoletion class example OpenFOAM
\*---------------------------------------------------------------------------*/

#include "fvCFD.H"
#include "interpolateXY.H"
#include "IFstream.H"
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

int main(int argc, char *argv[])
{ 
    
// arguments using example
// <) FOAM/applications/utilities/surface/surfaceTransformPoints/surfaceTransformPoints.C
    
argList::addNote
    (
        "p, T, rho (H) Earth"
    );
 
argList::addOption
    (
        "H",
        "scalar",
        "height of Earth"
    );
 argList::noBanner();

#include "setRootCase.H" //  инициализируется объект класса argList с именем args

 scalar H = 11111; // задаем значение высоты в метрах, на которой нужно найти термодинамические величины

 if (args.readIfPresent("H", H))
    {
       Info << "H = " << H << endl;
    }
 fileName atmospherePath = (args.path() + "/atmosphere");

 // Info << atmospherePath << endl; 

 IFstream in (atmospherePath); // создаем поток ввода из файла atmosphere

dictionary atmosphere (in); // инициализируем словарь

//Info << atmosphere << endl; // вывод словаря

// scalarField - (Field<scalar>) класс поля (наследуется от класса List) с расширенным функционалом
 
scalarField h (atmosphere.lookup("h")); // считываем массивы для h - высоты (м), T - температуры (K), p - давления (Па) и rho - плотности (кг/м^3) стандартной атмосферы Земли
scalarField T (atmosphere.lookup("T"));
scalarField p (atmosphere.lookup("p"));
scalarField rho(atmosphere.lookup("rho"));
 
 Info << "p = " << interpolateXY<scalar>(H, h, p) << endl; // функция линейной интерполяции, вывод значения давления с высоты H
 Info << "T = " << interpolateXY<scalar>(H, h, T) << endl; // вывод значения температуры с высоты H
 Info << "rho = " <<interpolateXY<scalar>(H, h, rho) << endl; // вывод значения плотности с высоты H

 //#include "createTime.H" // создается объект класса Time с именем runTime

    return 0;
}


// ************************************************************************* //
```

In [ ]:
%cat Make/files

In [ ]:
%cat Make/options

In [ ]:
# чистим и компилируем
!wclean
!wmake

In [ ]:
# смотрим содержимое файла атмосферы (табличные данные для интерполяции)
%cat atmosphere

In [ ]:
# описание программы
!Interpolation -help

In [ ]:
# запуск
!Interpolation

In [ ]:
# запускаем с аргументом (высота 100 метров)
!Interpolation -H 100

In [ ]:
# запуск в другой корневой папке (test)
!Interpolation -case test

In [ ]:
# пробуем запустить в папке, где нет файла атмосферы
!Interpolation -case ~/

## Примитивы OpenFOAM <a name="src/OpenFOAM"></a>

In [ ]:
# Путь к библиотеке OpenFOAM
!echo $WM_PROJECT_DIR
path = !echo $WM_PROJECT_DIR
path = path[0]

In [ ]:
# Базовые классы
!ls $path/src/OpenFOAM

In [ ]:
# Контейнеры
!ls $path/src/OpenFOAM/containers

In [ ]:
# Примитивы
!ls $path/src/OpenFOAM/primitives

In [ ]:
# Интерполяция
!ls $path/src/OpenFOAM/interpolations

In [ ]:
!ls $path/src/OpenFOAM/global

In [ ]:
# Матрицы
!ls $path/src/OpenFOAM/matrices

In [ ]:
# Сетки
!ls $path/src/OpenFOAM/meshes

## Класс Time <a name="Time"></a>

In [ ]:
# laplacianFoam
!ls $WM_PROJECT_DIR/applications/solvers/basic/laplacianFoam

In [ ]:
#!cat $WM_PROJECT_DIR/applications/solvers/basic/laplacianFoam/laplacianFoam.C

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | www.openfoam.com
     \\/     M anipulation  |
-------------------------------------------------------------------------------
    Copyright (C) 2011-2017 OpenFOAM Foundation
    Copyright (C) 2019 OpenCFD Ltd.
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

Application
    laplacianFoam

Group
    grpBasicSolvers

Description
    Laplace equation solver for a scalar quantity.

    \heading Solver details
    The solver is applicable to, e.g. for thermal diffusion in a solid.  The
    equation is given by:

    \f[
        \ddt{T}  = \div \left( D_T \grad T \right)
    \f]

    Where:
    \vartable
        T     | Scalar field which is solved for, e.g. temperature
        D_T   | Diffusion coefficient
    \endvartable

    \heading Required fields
    \plaintable
        T     | Scalar field which is solved for, e.g. temperature
    \endplaintable

\*---------------------------------------------------------------------------*/

#include "fvCFD.H"
#include "fvOptions.H"
#include "simpleControl.H"

// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

int main(int argc, char *argv[])
{
    argList::addNote
    (
        "Laplace equation solver for a scalar quantity."
    );

    #include "postProcess.H"

    #include "addCheckCaseOptions.H"
    #include "setRootCaseLists.H"
    #include "createTime.H"
    #include "createMesh.H"

    simpleControl simple(mesh);

    #include "createFields.H"

    // * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

    Info<< "\nCalculating temperature distribution\n" << endl;

    while (simple.loop())
    {
        Info<< "Time = " << runTime.timeName() << nl << endl;

        while (simple.correctNonOrthogonal())
        {
            fvScalarMatrix TEqn
            (
                fvm::ddt(T) - fvm::laplacian(DT, T)
             ==
                fvOptions(T)
            );

            fvOptions.constrain(TEqn);
            TEqn.solve();
            fvOptions.correct(T);
        }

        #include "write.H"

        runTime.printExecutionTime(Info);
    }

    Info<< "End\n" << endl;

    return 0;
}


// ************************************************************************* //
```

In [ ]:
#!cat $WM_PROJECT_DIR/src/OpenFOAM/include/createTime.H

```c++
Foam::Info<< "Create time\n" << Foam::endl;

Foam::Time runTime(Foam::Time::controlDictName, args);
```

FOAM::Time - Class to <font color='red'>control time</font> during OpenFOAM simulations that is also the top-level <font color='red'>objectRegistry</font>. (<font color='red'>cpuTime</font>)


[Time](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1Time.html) - doxygen

In [ ]:
#!cat Time/Time_OF.C

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | Copyright (C) 2011-2016 OpenFOAM Foundation
     \\/     M anipulation  |
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

Application
    Time_OF


Description
   Time class example OpenFOAM
\*---------------------------------------------------------------------------*/

#include "fvCFD.H"
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

int main(int argc, char *argv[])
{ 
argList::addNote
    (
        "Time class example OpenFOAM"
    );
 
#include "setRootCase.H" //  создается объект класса argList с именем args
#include "createTime.H" // создается объект класса Time с именем runTime


// .run - альтернатива .loop, с возможностью итерации времени по ходу цикла в любом месте
//while(runTime.run()){
// runTime++
 

 // общий цикл по времени
 // итеррируются значения времени + запускаются вспомогательные функции (например functionObjects)
 while(runTime.loop()){
     // clock
     // время выполнения кода
     Info << endl << "clock" << endl;
     Info << "runTime.elapsedClockTime : " << runTime.elapsedClockTime() << endl;

     // dimensioned (размерность объекта runTime)
     Info << endl << "dimensioned" << endl;
     Info << "runTime.dimensions() : "<< runTime.dimensions() << endl;
     // текущее значение
     Info << "runTime.value() : "<< runTime.value() << endl;
     // создаем новые объекты класса dimensionedScalar
     dimensionedScalar x;
     dimensionedScalar newTime (dimensionSet(0,0,1,0,0), 1);
     Info << "x : " << x << endl;
     Info << "newTime : " << newTime << endl;
     try {
       throw -1;
       Info << "runTime + x :" << runTime + x << endl;
     }
     catch (int a){
       Info << "OOOOOps!" << endl;
     }
     Info << "runTime + newTime :" << runTime + newTime << endl;

     // TimeState
     Info << endl << "TimeState"<< endl;
     // индекс текущей итерации
     Info << "runTime.timeIndex() : "<< runTime.timeIndex() << endl;
     // шаг по времени на текущей итерации (физическое время задачи)
     Info << "runTime.deltaT() : "<< runTime.deltaT() << endl;
     // шаг по времени на предведущей итерации (физическое время задачи)
     Info << "runTime.deltaT0() : "<< runTime.deltaT0() << endl;
     Info << "runTime.timeOutputValue() : "<< runTime.timeOutputValue() << endl;

     // TimePaths
     // Класс Time очень важный, от него наследуются многие другие классы (в том числе сетка и поля, которые зависят от времени)
     // Данные изменяемые со временем хранятся в папка, информация о пути которых содержится в самом классе Time
     // Есть методы для работы с путями
     Info << endl << "TimePaths"<< endl;
     Info << "runTime.times() : "<< runTime.times() << endl;
     if (runTime.value() != 0 and runTime.value() != runTime.deltaTValue())
       Info << "runTime.findClosestTime(2) : "<< runTime.findClosestTime(2) << endl;
     
     runTime.write();
     runTime.printExecutionTime(Info);
     sleep(runTime.deltaTValue());
   }

 // Time
 // Глобальные настройки расчета, которые содержатся в system/controlDict
 Info << endl << "Time" << endl;
 // Начальное время для расчета (папка с объектами)
 Info << "runTime.startTime() : "<< runTime.startTime() << endl;
 // Время окончания расчета
 Info << "runTime.endTime() : "<< runTime.endTime() << endl;
 // опция окончания расчета
 Info << "runTime.stopAt() : "<< runTime.stopAt() << endl;
 // опция адаптивного шага по времени
 Info << "runTime.isAdjustTimeStep() : "<< runTime.isAdjustTimeStep() << endl;
 // дополнительные динамические библиотеки
 Info << "runTime.libs() : "<< runTime.libs() << endl;

 // objectRegistry
 // Класс Time наследуется от класса objectRegistry, который позволяет иметь доступ к другим объектам
 // Иногда таким образом устроен интерфейс передачи данных внутри библиотек
 Info << endl << "objectRegistry" << endl;
 if (runTime.found("p"))
   volScalarField& p = runTime.lookupObjectRef<volScalarField>("p");
 else
   Info <<"no object with name p" <<endl;
   
 // IOobject
 
    return 0;
}


// ************************************************************************* //
```

In [ ]:
import os
os.environ['FOAM_USER_APPBIN'] = os.environ['FOAM_APPBIN'].replace("user1", os.environ['USER'])
#os.environ['FOAM_USER_APPBIN']
os.environ['PATH']=os.environ['PATH']+":"+os.environ['FOAM_USER_APPBIN']
#os.environ['PATH']
os.environ['FOAM_USER_LIBBIN'] = os.environ['FOAM_LIBBIN'].replace("user1", os.environ['USER'])
#os.environ['FOAM_USER_LIBBIN']

In [ ]:
cd ..

In [ ]:
!wclean Time/
!wmake Time/

In [ ]:
!Time_OF

In [ ]:
!rm -r Time/test/3/
!rm -r Time/test/1.5/
!ls Time/test

In [ ]:
!ls Time/test/system

In [ ]:
#!cat Time/test/system/controlDict

```c++
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2012                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "system";
    object      controlDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

startFrom       startTime; //latestTime;

startTime       0;

stopAt          endTime;

endTime         3;

deltaT          1.5;

writeControl    runTime;

writeInterval   $deltaT;

purgeWrite      0;

writeFormat     ascii;

writePrecision  16;

writeCompression off;

timeFormat      general;

timePrecision   16;

runTimeModifiable true;


// ************************************************************************* //
```

In [ ]:
!Time_OF -case Time/test/

In [ ]:
!Time_OF -case Time/test/

In [ ]:
ls Time/test/

In [ ]:
ls Time/test/3

In [ ]:
ls Time/test/3/uniform/

In [ ]:
!cat Time/test/3/uniform/time

## Формат сетки OpenFOAM <a name="mesh"></a>

[Wolfdynamics](../books/wd_mesh.pdf) p. 12 Структурированные vs Неструктурированные сетки

[CFD_OF](../books/darwish.pdf) p. 200 (175)

[UserGuide](../books/OFug.pdf) p.139


In [ ]:
ls Simple_mesh/

Узел (point, vertex) - точка сетки с координатами (x,y,z)

Ребро (edge) - пара узлов

Грань (face) - список узлов (больше 2х)

Ячейка (cell) - список граней (больше 3х)

<img src="Simple_mesh/points.png" width="500" >

<img src="Simple_mesh/simple_mesh_wireframe.png" width="500" >

<img src="Simple_mesh/simple_mesh.png" width="500" >

In [ ]:
#!cat Simple_mesh/system/blockMeshDict

```c++
scale   1;

vertices
(
    (0 0 0)
    (2 0 0)
    (2 1 0)
    (0 1 0)
    (0 0 1)
    (2 0 1)
    (2 1 1)
    (0 1 1)
);

blocks
(
    hex (0 1 2 3 4 5 6 7) (2 1 1) simpleGrading (1 1 1)
);
```


In [ ]:
!ls Simple_mesh/constant/polyMesh # OpenFOAM mesh
# points - список узлов
# faces - список граней
# owner - список владельцев
# neighbour - список соседей
# boundary - описание границы

In [ ]:
#!cat Simple_mesh/constant/polyMesh/points # список узлов (точек)

```c++
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2012                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       vectorField;
    location    "constant/polyMesh";
    object      points;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //


12 // список из 12 элементов
(
(0 0 0) // 0 x,y,z координаты точки
(1 0 0) // 1
(2 0 0) // 2
(0 1 0) // 3
(1 1 0) // 4
(2 1 0) // 5
(0 0 1) // 6
(1 0 1) // 7
(2 0 1) // 8
(0 1 1) // 9
(1 1 1) // 10
(2 1 1) // 11
)
```
<img src="Simple_mesh/points.png" width="500">

In [ ]:
#!cat Simple_mesh/constant/polyMesh/faces # список граней

```c++
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2012                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       faceList;
    location    "constant/polyMesh";
    object      faces;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //


11 // список из 11 элементов
(
4(1 4 10 7) // массив точек (номера из points)
4(3 9 10 4)
4(4 10 11 5)
4(0 6 9 3)
4(2 5 11 8)
4(0 1 7 6)
4(1 2 8 7)
4(0 3 4 1)
4(1 4 5 2)
4(6 7 10 9)
4(7 8 11 10)
)


// ************************************************************************* //
```

у каждой грани всегда <font color='red'>2</font> стороны!

одной стороне соответствует ячейка владельца (<font color='blue'>owner</font>)

другой стороне соответствует ячейка соседа (<font color='red'>neighbour</font>)

<img src="Simple_mesh/simple_mesh.png" width="500">

In [ ]:
#!cat Simple_mesh/constant/polyMesh/owner # список номеров ячеек владельцев

```c++
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2012                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       labelList;
    note        "nPoints:12  nCells:2  nFaces:11  nInternalFaces:1";
    location    "constant/polyMesh";
    object      owner;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //


11 // массив из 11 элементов
(
0 // номер ячейки владельца (owner) для грани 0 (из faces)
0 // номер ячейки владельца (owner) для грани 1 (из faces)
1 // номер ячейки владельца (owner) для грани 2 (из faces)
0
1
0
1
0
1
0
1
)


// ************************************************************************* //
```
<img src="Simple_mesh/simple_mesh_wireframe.png" width="500">

In [ ]:
#!cat Simple_mesh/constant/polyMesh/neighbour # список соседей

```c++
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2012                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       labelList;
    note        "nPoints:12  nCells:2  nFaces:11  nInternalFaces:1";
    location    "constant/polyMesh";
    object      neighbour;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

1 // список из 1го элемента
(
1 // номер ячейки соседа (neighbour) для грани 0 (из faces)
)
    
```
<img src="Simple_mesh/simple_mesh_wireframe.png" width="500">

In [ ]:
#!cat Simple_mesh/constant/polyMesh/boundary # описание границы

```c++
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2012                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       polyBoundaryMesh;
    location    "constant/polyMesh";
    object      boundary;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

3 // список из 3 элементов
(
    movingWall // название границы (patch - массив граней)
    {
        type            wall; // тип границы
        inGroups        1(wall);
        nFaces          2; // кол-во граней в массиве
        startFace       1; // стартовый номер грани
    }
    fixedWalls
    {
        type            wall;
        inGroups        1(wall);
        nFaces          4;
        startFace       3;
    }
    frontAndBack
    {
        type            empty;
        inGroups        1(empty);
        nFaces          4;
        startFace       7;
    }
)

// ************************************************************************* //
```

## Заключение <a name="finish"></a>

[Познакомились с работой на вычислительном кластере](#slurm)

[Познакомились с базовыми примитивами OpenFOAM](#Interpolation)

[Узнали, что примитивов даже больше чем нужно](#src/OpenFOAM)

[Узнали про класс Time](#Time)

[Рассмотрели формат сетки OpenFOAM](#mesh)

## В следующей серии ...

Конечнообъемная Сетка, поля и динамические сетки!

<img src="img/mesh.jpeg" width="400">
<img src="img/field.jpg" width="400">
<img src="img/dynamic_mesh.jpg" width="400">
